Endspoints:

In [1]:
import pandas as pd
import gzip
import json
import ast
import re
import random

In [16]:
ruta_archivo = 'steam_games.csv'

# Usa la función read_csv para abrir el archivo CSV
df = pd.read_csv(ruta_archivo)

In [17]:
ruta_archivo = 'user_reviews.csv'

# Usa la función read_csv para abrir el archivo CSV
df2 = pd.read_csv(ruta_archivo)

In [18]:
ruta_archivo = 'user_items.csv'

# Usa la función read_csv para abrir el archivo CSV
df3 = pd.read_csv(ruta_archivo)

1- def PlayTimeGenre( genero : str ): Debe devolver año con mas horas jugadas para dicho género.

    Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

In [19]:
def PlayTimeGenre(genero: str):
    df['user_id'] = df['user_id'].astype(str)
    df3['user_id'] = df3['user_id'].astype(str)
    # Fusionar ambos DataFrames en base a la columna 'user_id'
    df_combinado = df.merge(df3, left_on='user_id', right_on='user_id', how='inner')
    df_filtrado = df_combinado[df_combinado['Genres'].str.contains(genero, case=False, na=False)]

    if df_filtrado.empty:
        return {}  

    
    agrupado = df_filtrado.groupby('Release_Year')['playtime_forever'].sum()
    max_year = agrupado.idxmax()
    resultado = {"Año de lanzamiento con más horas jugadas para Género " + genero: max_year}

    return resultado

In [20]:
PlayTimeGenre('Indie')


{'Año de lanzamiento con más horas jugadas para Género Indie': 2014}

2-def UserForGenre( genero : str ): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

    Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [8]:
def UserForGenre(genero: str):
    
    df['user_id'] = df['user_id'].astype(str)
    df3['user_id'] = df3['user_id'].astype(str)
    
    # Fusionar ambos DataFrames en base a la columna 'user_id'
    df_combinado = df.merge(df3, left_on='user_id', right_on='user_id', how='inner')

    df_filtrado = df_combinado[df_combinado['Genres'].str.contains(genero, case=False, na=False)]
    if df_filtrado.empty:
        return {}  
    agrupado = df_filtrado.groupby(['user_id', 'Release_Year'])['playtime_forever'].sum().reset_index()
    max_user = agrupado.groupby('user_id')['playtime_forever'].sum().idxmax()
    df_max_user = agrupado[agrupado['user_id'] == max_user]
    lista_acumulacion = df_max_user.rename(columns={'Release_Year': 'Año', 'playtime_forever': 'Horas'}).to_dict('records')
    resultado = {
        "Usuario con más horas jugadas para el Género " + genero: max_user,
        "Horas jugadas": lista_acumulacion
    }

    return resultado

In [9]:
UserForGenre('Indie')

{'Usuario con más horas jugadas para el Género Indie': '305010',
 'Horas jugadas': [{'user_id': '305010', 'Año': 2014, 'Horas': 59519}]}

3-def UsersRecommend( año : int ): Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)

In [21]:


def UsersRecommend(año: int):
    # filas para el año  y donde recommend es False
    juegos_no_recomendados = df2[(df2['recommend'] == True)]
    
    # muestra aleatoria de df3
    sample_size = 90000  
    df3_sample = df3.sample(n=sample_size, random_state=42)
    
    
    juegos_no_recomendados = juegos_no_recomendados.merge(df3_sample[['item_id', 'item_name']], on='item_id', how='left')
    conteo_juegos = juegos_no_recomendados['item_name'].value_counts().reset_index()
    conteo_juegos.columns = ['item_name', 'count']
    top_juegos_no_recomendados = conteo_juegos.sort_values(by='count', ascending=True).head(3)
    
    resultado = [{"Puesto {}: {}".format(i+1, juego)} for i, juego in enumerate(top_juegos_no_recomendados['item_name'])]
    
    return resultado


In [22]:
UsersRecommend(2014)

[{'Puesto 1: Earthworm Jim 3D'},
 {'Puesto 2: Organic Panic'},
 {'Puesto 3: 3D Ultra Minigolf Adventures Deluxe'}]

4-def UsersNotRecommend( año : int ): Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)

   Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [10]:
def UsersNotRecommend(año: int):
     #filas para el año dado y donde recommend es False
    juegos_no_recomendados = df2[(df2['recommend'] == False)]
    
    juegos_no_recomendados = juegos_no_recomendados.merge(df3[['item_id', 'item_name']], on='item_id', how='left')
    conteo_juegos = juegos_no_recomendados['item_name'].value_counts().reset_index()
    conteo_juegos.columns = ['item_name', 'count']
    
    top_juegos_no_recomendados = conteo_juegos.sort_values(by='count', ascending=True).head(3)
    
    
    resultado = [{"Puesto {}: {}".format(i+1, juego)} for i, juego in enumerate(top_juegos_no_recomendados['item_name'])]
    
    return resultado



In [11]:
UsersNotRecommend(2011)


[{'Puesto 1: Avaris 2'}, {'Puesto 2: World of Subways 1 – The Path'}, {'Puesto 3: Shrooms'}]


5-def sentiment_analysis( año : int ): Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.

   Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [12]:
def sentiment_analysis(anio: int):
    # Filtra el DataFrame para obtener solo las filas correspondientes al año especificado
    df_filtrado = df2[df2['Year'] == anio]

    # Contabiliza la cantidad de registros en cada categoría de análisis de sentimiento
    categorias_sentimiento = df_filtrado['sentiment_analysis'].value_counts()

    # Crea un diccionario con el formato deseado
    resultado = {
        'Negativo': categorias_sentimiento.get(0, 0),  # Si no hay registros negativos, se establece en 0
        'Neutral': categorias_sentimiento.get(1, 0),   # Si no hay registros neutrales, se establece en 0
        'Positivo': categorias_sentimiento.get(2, 0)   # Si no hay registros positivos, se establece en 0
    }

    return resultado

In [13]:
# Ejemplo de uso de la función
anio = 2014  # Cambia el año según tus necesidades
resultado = sentiment_analysis(anio)
print(resultado)

{'Negativo': 1830, 'Neutral': 13358, 'Positivo': 6923}
